In [44]:
import numpy as np
import pandas as pd
import unicodedata
import re
import tensorflow as tf 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.initializers import GlorotUniform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam


In [45]:
data = pd.read_csv('output.csv')


In [47]:
data.head()

,province,district,street,price_unit,price_value,price_absolute,area,type,front,room,toilet,floor,attributes,Chung cư,Dòng tiền,Không rõ,Kinh doanh,Mặt phố,Thang máy,Ô tô
0,1,5,Mặt phố C1 TT Nghĩa Tân,1,5.50,"5,500.00",70.00,1,7.00,2,2,1,"Ô tô, Kinh doanh, Dòng tiền",0,1,0,1,0,0,1
1,1,5,Yên Hoà,1,8.58,"8,580.00",45.00,1,5.00,5,4,4,Ô tô,0,0,0,0,0,0,1
2,1,5,Cầu Giấy,1,8.65,"8,650.00",43.00,1,3.60,6,1,5,Không rõ,0,0,1,0,0,0,0
3,1,5,Hồ Tùng Mậu,1,28.50,"28,500.00",100.00,1,6.90,29,0,8,"Ô tô, Dòng tiền, Thang máy",0,1,0,0,0,1,1
4,1,5,Cầu Giấy,1,8.90,"8,900.00",35.00,1,3.40,6,2,5,Không rõ,0,0,1,0,0,0,0


In [48]:

# 1. Loại bỏ khoảng trắng và điền NaN
data['attributes'] = data['attributes'].str.strip()
data['attributes'] = data['attributes'].fillna('Không rõ')  # Loại bỏ inplace=True

# 2. Mã hóa các thuộc tính
attributes_dummies = data['attributes'].str.get_dummies(sep=', ')

# 3. Ghép các cột đã mã hóa vào DataFrame gốc
data = pd.concat([data, attributes_dummies], axis=1)

# 4. Kiểm tra kết quả
print(data.head())


   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  ...  Mặt phố  Thang máy  Ô tô  \
0        5,500.00  70.00     1   7.00     2  ...        0          0     1   
1        8,580.00  45.00     1   5.00     5  ...        0          0     1   
2        8,650.00  43.00     1   3.60     6  ...        0          0     0   
3       28,500.00 100.00     1   6.90    29  ...        0          1     1   
4        8,900.00  35.00     1   3.40     6  ...        0          0     0   

   Chung cư  Dòng tiền  Không rõ  Kinh doanh  Mặt phố  Thang máy  Ô tô  
0      

In [49]:
 # Hàm chuẩn hóa tên đường
def normalize_street_name(street_name):
    # Loại bỏ dấu tiếng Việt
    street_name = unicodedata.normalize('NFKD', street_name).encode('ascii', 'ignore').decode('utf-8')
    # Chuyển thành chữ thường
    street_name = street_name.lower()
    # Loại bỏ các ký tự đặc biệt và khoảng trắng thừa
    street_name = re.sub(r'[^a-z0-9\s]', '', street_name)
    street_name = re.sub(r'\s+', ' ', street_name).strip()
    return street_name

# Áp dụng hàm chuẩn hóa lên cột 'street'
data['street_normalized'] = data['street'].apply(normalize_street_name)

# Thực hiện one-hot encoding sau khi chuẩn hóa
street_encoded = pd.get_dummies(data['street_normalized'], prefix='street')

# Nối cột mới với DataFrame gốc
data = pd.concat([data, street_encoded], axis=1)

# Xóa cột 'street_normalized' nếu không cần thiết nữa
data.drop(columns=['street_normalized'], inplace=True)

# Kiểm tra lại dữ liệu sau khi xử lý
print(data.head())

   province  district                   street  price_unit  price_value  \
0         1         5  Mặt phố C1 TT Nghĩa Tân           1         5.50   
1         1         5                  Yên Hoà           1         8.58   
2         1         5                 Cầu Giấy           1         8.65   
3         1         5              Hồ Tùng Mậu           1        28.50   
4         1         5                 Cầu Giấy           1         8.90   

   price_absolute   area  type  front  room  ...  street_truong cong giai  \
0        5,500.00  70.00     1   7.00     2  ...                    False   
1        8,580.00  45.00     1   5.00     5  ...                    False   
2        8,650.00  43.00     1   3.60     6  ...                    False   
3       28,500.00 100.00     1   6.90    29  ...                    False   
4        8,900.00  35.00     1   3.40     6  ...                    False   

   street_tt van cong mai dich street_tt van hoa nghe thuat mai dich  \
0             

In [50]:
data = data.drop(columns=['street'])


In [51]:
data = data.drop(columns=['attributes'])


In [52]:
data = data.drop(columns=['price_value'])


In [53]:
X = data.drop('price_absolute', axis=1)
y = data['price_absolute']

In [54]:
X.info


<bound method DataFrame.info of      province  district  price_unit     area  type  front  room  toilet  \
0           1         5           1    70.00     1   7.00     2       2   
1           1         5           1    45.00     1   5.00     5       4   
2           1         5           1    43.00     1   3.60     6       1   
3           1         5           1   100.00     1   6.90    29       0   
4           1         5           1    35.00     1   3.40     6       2   
5           1         5           1    60.00     1   3.00     2       1   
6           1         5           1   130.00     1   6.90     2       0   
7           1         5           1   130.00     1   6.90     0       0   
8           1         5           1    77.00     1   7.00     7       3   
9           1         5           1    72.00     1   6.00     8       4   
10          1         5           1    48.00     1   3.50     5       1   
11          1         5           1   212.00     1   7.60     7     

In [55]:
X.shape


(551, 173)

In [56]:
X

,province,district,price_unit,area,type,front,room,toilet,floor,Chung cư,...,street_truong cong giai,street_tt van cong mai dich,street_tt van hoa nghe thuat mai dich,street_tu mo,street_uong 800a,street_vo chi cong,street_xuan thuy,street_xx cau giay,street_xx tran thai tong,street_yen hoa
0,1,5,1,70.00,1,7.00,2,2,1,0,...,False,False,False,False,False,False,False,False,False,False
1,1,5,1,45.00,1,5.00,5,4,4,0,...,False,False,False,False,False,False,False,False,False,True
2,1,5,1,43.00,1,3.60,6,1,5,0,...,False,False,False,False,False,False,False,False,False,False
3,1,5,1,100.00,1,6.90,29,0,8,0,...,False,False,False,False,False,False,False,False,False,False
4,1,5,1,35.00,1,3.40,6,2,5,0,...,False,False,False,False,False,False,False,False,False,False
5,1,5,1,60.00,1,3.00,2,1,1,0,...,False,False,False,False,False,False,False,False,False,False
6,1,5,1,130.00,1,6.90,2,0,2,0,...,False,False,False,False,False,False,False,False,False,False
7,1,5,1,130.00,1,6.90,0,0,2,0,...,False,False,False,False,False,False,False,False,False,False
8,1,5,1,77.00,1,7.00,7,3,4,0,...,False,False,False,False,False,False,False,False,False,False
9,1,5,1,72.00,1,6.00,8,4,4,0,...,False,False,False,False,False,False,False,False,False,False


In [57]:
y.shape


(551,)

In [58]:
from sklearn.model_selection import train_test_split


In [59]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=101)


In [60]:
X_train 

,province,district,price_unit,area,type,front,room,toilet,floor,Chung cư,...,street_truong cong giai,street_tt van cong mai dich,street_tt van hoa nghe thuat mai dich,street_tu mo,street_uong 800a,street_vo chi cong,street_xuan thuy,street_xx cau giay,street_xx tran thai tong,street_yen hoa
297,1,5,1,48.00,1,4.30,2,6,8,0,...,False,False,False,False,True,False,False,False,False,False
323,1,5,1,62.00,1,4.00,5,0,5,0,...,False,False,False,False,False,False,False,False,False,False
459,1,5,1,45.00,1,4.00,5,1,4,0,...,False,False,False,False,False,False,False,False,False,False
209,1,5,1,95.00,1,6.00,3,2,1,1,...,False,False,False,False,False,False,False,False,False,False
143,1,5,1,83.00,1,7.70,10,0,5,1,...,False,False,False,False,False,False,False,False,False,False
468,1,5,1,94.00,1,7.50,10,7,6,0,...,False,False,False,False,False,False,False,False,False,False
225,1,5,1,48.00,1,4.00,9,0,5,0,...,False,False,False,False,False,False,False,False,False,False
318,1,5,1,60.00,1,4.00,7,3,5,0,...,False,False,False,False,False,False,False,False,False,False
216,1,5,1,77.00,1,6.20,10,4,4,0,...,False,False,False,False,False,False,False,False,False,False
53,1,5,1,30.00,1,4.80,5,1,5,0,...,False,False,False,False,False,False,False,False,False,False


In [61]:
X_test


,province,district,price_unit,area,type,front,room,toilet,floor,Chung cư,...,street_truong cong giai,street_tt van cong mai dich,street_tt van hoa nghe thuat mai dich,street_tu mo,street_uong 800a,street_vo chi cong,street_xuan thuy,street_xx cau giay,street_xx tran thai tong,street_yen hoa
220,1,5,1,48.00,1,3.80,2,3,6,0,...,False,False,False,False,False,False,False,False,False,False
18,1,5,1,63.00,1,5.00,12,1,7,0,...,False,False,False,False,False,False,False,False,False,False
391,1,5,1,31.00,1,3.00,7,4,5,0,...,False,False,False,False,False,False,False,False,False,False
141,1,5,1,103.00,1,4.20,0,0,7,0,...,False,False,False,False,False,False,False,False,False,False
354,1,5,1,57.00,1,6.30,0,0,5,0,...,False,False,False,False,False,False,False,False,False,False
116,1,5,1,48.00,1,3.40,0,0,1,0,...,False,False,False,False,False,False,False,False,False,False
313,1,5,1,61.00,1,3.30,22,11,6,0,...,False,False,False,False,False,False,False,False,False,False
138,1,5,1,65.00,1,3.40,2,0,4,0,...,False,False,False,False,False,False,False,False,False,False
515,1,5,1,81.00,1,5.50,0,0,4,0,...,False,False,False,False,False,False,False,False,False,False
360,1,5,1,49.00,1,4.20,9,9,5,0,...,False,False,False,False,False,False,False,False,False,False


In [62]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()


In [63]:
X_train = scaler.fit_transform(X_train)


In [64]:
X_test = scaler.transform(X_test)


In [65]:
X_train.shape


(385, 173)

In [66]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [67]:
X_test.shape


(166, 173)

In [88]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from keras_tuner import RandomSearch

# Hàm tạo mô hình
def build_model(hp):
    model = Sequential()

    # Sử dụng Hyperparameters để điều chỉnh số lượng neurons và dropout
    for i in range(4):  # Thêm 4 layers
        model.add(Dense(units=hp.Int('units_' + str(i), min_value=16, max_value=64, step=16),
                        activation='relu',
                        kernel_regularizer=l2(hp.Float('l2_' + str(i), 0.001, 0.1, step=0.01))))
        model.add(Dropout(hp.Float('dropout_' + str(i), 0.1, 0.5, step=0.1)))
    
    # Lớp đầu ra
    model.add(Dense(1))

    # Sử dụng hyperparameter để điều chỉnh learning rate của Adam
    learning_rate = hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    return model


In [89]:
from keras_tuner import RandomSearch

# Khởi tạo Keras Tuner
tuner = RandomSearch(
    build_model,  # Hàm tạo mô hình
    objective='val_loss',  # Tối ưu hóa dựa trên validation loss
    max_trials=40,  # Số lần thử nghiệm với các hyperparameter khác nhau
    executions_per_trial=2,  # Số lần chạy mỗi thử nghiệm
    directory='my_dir',  # Thư mục để lưu kết quả
    project_name='house_price_prediction'  # Tên dự án
)


Reloading Tuner from my_dir\house_price_prediction\tuner0.json


In [90]:
# Bắt đầu tìm kiếm hyperparameters tốt nhất
tuner.search(X_train, y_train, epochs=100, validation_data=(X_test, y_test))

# Lấy ra mô hình tốt nhất
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"Best number of neurons in first layer: {best_hps.get('units_0')}")
print(f"Best dropout rate in first layer: {best_hps.get('dropout_0')}")
print(f"Best learning rate: {best_hps.get('learning_rate')}")
print(f"Best L2 rate :{best_hps.get('l2_0')}")


Best number of neurons in first layer: 64
Best dropout rate in first layer: 0.1
Best learning rate: 0.007354966228502024
Best L2 rate :0.07100000000000001


In [91]:
# Lấy mô hình tốt nhất từ Keras Tuner và huấn luyện lại mô hình
best_model = tuner.hypermodel.build(best_hps)

In [92]:

# Huấn luyện lại mô hình tốt nhất từ Keras Tuner
history = best_model.fit(
    X_train, y_train, 
    epochs=200,  # Số lượng epochs tối đa
    batch_size=128,  # Sử dụng batch_size = 32
    validation_data=(X_test, y_test),  # Validation data để đánh giá mô hình
    verbose=1  # Hiển thị chi tiết quá trình huấn luyện
)

# Dự đoán trên tập kiểm tra
y_test_pred = best_model.predict(X_test)

# Tính toán các chỉ số đánh giá: MSE, MAE, R² và RMSE
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np  # Thêm numpy để tính căn bậc hai

# Tính các chỉ số
mse = mean_squared_error(y_test, y_test_pred)
mae = mean_absolute_error(y_test, y_test_pred)
r2 = r2_score(y_test, y_test_pred)
rmse = np.sqrt(mse)  # Tính RMSE bằng cách lấy căn bậc hai của MSE

# In kết quả
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')
print(f'RMSE: {rmse}')


Epoch 1/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 61ms/step - loss: 4964153856.0000 - val_loss: 22826780672.0000
Epoch 2/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4698378240.0000 - val_loss: 22826446848.0000
Epoch 3/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2887174400.0000 - val_loss: 22825316352.0000
Epoch 4/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 4782721024.0000 - val_loss: 22821464064.0000
Epoch 5/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 3435944448.0000 - val_loss: 22808762368.0000
Epoch 6/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4982074368.0000 - val_loss: 22773137408.0000
Epoch 7/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 2759769344.0000 - val_loss: 22684575744.0000
Epoch 8/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 2830282752.0000 - val_loss: 22494191616.0000
Epoch 9/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 4690855424.0000 - val_loss: 22118156288.0000
Epoch 10/200
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 3158245888.000

In [93]:
y_test_pred = best_model.predict(X_test)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [94]:
y_test

220      17,500.00
18       16,500.00
391       7,550.00
141      44,000.00
354      14,500.00
116      26,500.00
313      13,500.00
138      22,500.00
515      25,500.00
360       8,600.00
236      16,700.00
8        23,600.00
437      13,900.00
505      22,000.00
39        8,600.00
277      72,000.00
55        5,550.00
171      15,200.00
228       5,950.00
413      31,500.00
269      25,500.00
290       9,800.00
521      16,000.00
83       22,000.00
48      195,000.00
540      20,500.00
156      39,500.00
317      13,200.00
147      36,500.00
377       5,750.00
114      18,800.00
186      15,200.00
118       6,050.00
32      142,000.00
117      71,000.00
291       9,300.00
183      55,000.00
400      16,500.00
427      32,000.00
218     120,000.00
486       5,200.00
0         5,500.00
107      27,300.00
4         8,900.00
139      19,700.00
535      12,700.00
283      35,000.00
265       6,980.00
179      24,000.00
450      51,000.00
100      12,000.00
255      15,900.00
359      14,

In [95]:
import pandas as pd

# Chuyển đổi giá trị dự đoán và giá trị thực thành DataFrame để tiện so sánh
df_comparison = pd.DataFrame({'Actual': y_test.values, 'Predicted': y_test_pred.flatten()})

# Định dạng các số bằng cách giới hạn số chữ số thập phân
pd.options.display.float_format = '{:,.2f}'.format

# Hiển thị kết quả
print(df_comparison)


          Actual  Predicted
0      17,500.00  19,886.44
1      16,500.00  10,589.34
2       7,550.00   9,519.44
3      44,000.00  23,975.73
4      14,500.00  11,123.41
5      26,500.00  28,472.52
6      13,500.00  25,302.67
7      22,500.00  13,753.91
8      25,500.00  10,830.68
9       8,600.00  10,967.25
10     16,700.00  20,990.33
11     23,600.00  15,884.78
12     13,900.00   9,287.44
13     22,000.00  27,776.69
14      8,600.00  12,301.10
15     72,000.00  31,412.06
16      5,550.00   9,518.08
17     15,200.00  19,428.22
18      5,950.00   9,898.96
19     31,500.00  15,717.06
20     25,500.00  41,300.54
21      9,800.00  27,035.13
22     16,000.00  14,989.60
23     22,000.00  17,966.32
24    195,000.00  47,629.06
25     20,500.00  18,638.98
26     39,500.00  31,463.73
27     13,200.00  11,605.08
28     36,500.00  17,874.45
29      5,750.00  21,818.29
30     18,800.00  14,528.00
31     15,200.00  28,897.48
32      6,050.00  21,251.52
33    142,000.00  66,108.68
34     71,000.00  23

In [96]:
# Tính sai số tuyệt đối
df_comparison['Absolute Error'] = abs(df_comparison['Actual'] - df_comparison['Predicted'])

# Sắp xếp DataFrame theo sai số từ cao xuống thấp
df_comparison_sorted = df_comparison.sort_values(by='Absolute Error', ascending=False)

# Hiển thị 10 kết quả có sai số lớn nhất
print(df_comparison_sorted.head(10))


          Actual  Predicted  Absolute Error
158 1,550,000.00 788,916.94      761,083.06
60    320,000.00  22,190.72      297,809.28
64    350,000.00  77,035.68      272,964.32
65    285,000.00  67,187.96      217,812.04
119   225,000.00  48,604.00      176,396.00
24    195,000.00  47,629.06      147,370.94
39    120,000.00  17,399.66      102,600.34
33    142,000.00  66,108.68       75,891.32
115    97,000.00  24,979.91       72,020.09
56     86,000.00  22,502.81       63,497.19
